**Set up colab for ASL data from kaggel**

In [ ]:
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download grassknoted/asl-alphabet

In [ ]:
! unzip asl-alphabet.zip

**Import needed libraries**

In [ ]:
import numpy as np
import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, GridSearchCV
import cv2
import keras
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization,Activation,MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras import regularizers
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [ ]:
train_dir = 'asl_alphabet_train//asl_alphabet_train'
test_dir = 'asl_alphabet_test//asl_alphabet_test'
labels_dict = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11,
               'M': 12,
               'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23,
               'Y': 24,
               'Z': 25, 'space': 26, 'del': 27, 'nothing': 28}

In [ ]:
def load_train_data():
    Y_train = []
    X_train = []
    size = 64, 64
    #number_of_images_per_folder = 0
    images_per_folder = 0
    print("LOADING DATA FROM : ", end="")
    for folder in os.listdir(train_dir):
        print(folder, end=' | ')
        for image in os.listdir(train_dir + "/" + folder):
            if images_per_folder == 2000:
              images_per_folder = 0
              break
            # read image
            temp_img = cv2.imread(train_dir + '/' + folder + '/' + image)
            # resize image
            temp_img = cv2.resize(temp_img, size)
            #load converted classes
            Y_train.append(labels_dict[folder])
            X_train.append(temp_img)
            images_per_folder  = images_per_folder +1
    #convert X_train to numpy
    X_train = np.array(X_train)
    #normalize the pixels of X_train
    X_train = X_train.astype('float32')/255.0
    #convert Y_train to numpy
    Y_train = np.array(Y_train)
    print()
    print('Loaded', len(X_train), 'images for training,', 'Train data shape =', X_train.shape)

    return X_train, Y_train


In [ ]:
def load_test_data():
    labels = []
    X_test = []
    size = 64, 64
    for image in os.listdir(test_dir):
        # read image
        temp_img = cv2.imread(test_dir + '/'+ image)
        # resize image
        temp_img = cv2.resize(temp_img, size)
        # load converted classes
        labels.append(labels_dict[image.split('_')[0]])
        X_test.append(temp_img)
    #convert X_test to numpy
    X_test = np.array(X_test)
    #normalize pixels of X_test
    X_test = X_test.astype('float32')/255.0
    #convert Y_test to numpy
    Y_test = np.array(labels)
    print('Loaded', len(X_test), 'images for testing,', 'Test data shape =', X_test.shape)

    return X_test, Y_test

**Load data with Gray**

In [ ]:
X_train, Y_train =  load_train_data()

LOADING DATA FROM : D | Z | V | del | space | P | U | nothing | T | B | C | A | F | J | L | E | G | S | Y | R | Q | M | N | X | K | I | O | H | W | 
Loaded 58000 images for training, Train data shape = (58000, 64, 64, 3)


In [ ]:
X_test, Y_test = load_test_data()

Loaded 28 images for testing, Test data shape = (28, 64, 64, 3)


**Create first CNN model**


In [ ]:
def create_model():
    
    model = Sequential()
    
    model.add(Conv2D(16, kernel_size = [3,3], padding = 'same', activation = 'relu', input_shape = (64,64,3)))
    model.add(Conv2D(32, kernel_size = [3,3], padding = 'same', activation = 'relu'))
    model.add(MaxPool2D(pool_size = [3,3]))
    
    model.add(Conv2D(32, kernel_size = [3,3], padding = 'same', activation = 'relu'))
    model.add(Conv2D(64, kernel_size = [3,3], padding = 'same', activation = 'relu'))
    model.add(MaxPool2D(pool_size = [3,3]))

    model.add(Conv2D(128, kernel_size = [3,3], padding = 'same', activation = 'relu', input_shape = (64,64,3)))
    model.add(Conv2D(256, kernel_size = [3,3], padding = 'same', activation = 'relu'))
    model.add(MaxPool2D(pool_size = [3,3]))
    model.add(BatchNormalization())
    model.add(Flatten())
    
    model.add(Dropout(rate = 0.5))
    model.add(Dense(512, activation = 'relu',kernel_regularizer = regularizers.l2(0.001)))
    model.add(Dense(29, activation = 'softmax'))
    model.summary()   
    return model

**Crete second CNN model**

In [ ]:
def create_model_2():
     model = Sequential()
     model.add(Conv2D(32, (5, 5), input_shape=(64, 64, 3)))
     model.add(Activation('relu'))
     model.add(MaxPooling2D((2, 2)))

     model.add(Conv2D(64, (3, 3)))
     model.add(Activation('relu'))
     model.add(MaxPooling2D((2, 2)))

     model.add(Conv2D(64, (3, 3)))
     model.add(Activation('relu'))
     model.add(MaxPooling2D((2, 2)))
     model.add(BatchNormalization())
     model.add(Flatten())

     model.add(Dense(128, activation='relu'))

     model.add(Dense(29, activation='softmax'))

     model.summary()
     return model

**Description of model 2**

In [ ]:
model_2 = create_model_2()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 60, 60, 32)        2432      
                                                                 
 activation_3 (Activation)   (None, 60, 60, 32)        0         
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 30, 30, 32)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 28, 28, 64)        18496     
                                                                 
 activation_4 (Activation)   (None, 28, 28, 64)        0         
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                  

**Description of model 1**

In [ ]:
model = create_model()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 64, 64, 16)        448       
                                                                 
 conv2d_13 (Conv2D)          (None, 64, 64, 32)        4640      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 21, 21, 32)       0         
 2D)                                                             
                                                                 
 conv2d_14 (Conv2D)          (None, 21, 21, 32)        9248      
                                                                 
 conv2d_15 (Conv2D)          (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 7, 7, 64)         0         
 g2D)                                                 

**Compile fisrt model**

In [ ]:
model.compile(optimizer='adam',loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])

**Compile second model**

In [ ]:
model_2.compile(optimizer='adam',loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])

**Fit first model with 5 epochs**

In [ ]:
epochs = 5
ThisModel = model.fit(X_train,Y_train,epochs = epochs, batch_size=64,verbose = 1)

Epoch 1/5
907/907 [==============================] - 749s 825ms/step - loss: 1.4935 - accuracy: 0.5826
Epoch 2/5
907/907 [==============================] - 736s 812ms/step - loss: 0.3193 - accuracy: 0.9408
Epoch 3/5
907/907 [==============================] - 736s 812ms/step - loss: 0.2139 - accuracy: 0.9663
Epoch 4/5
907/907 [==============================] - 742s 818ms/step - loss: 0.1768 - accuracy: 0.9751
Epoch 5/5
907/907 [==============================] - 728s 803ms/step - loss: 0.1566 - accuracy: 0.9795


**Fit second model with 5 epochs**

In [ ]:
ThisModel_2 = model_2.fit(X_train, Y_train,epochs=5,batch_size=64,verbose=1)

Epoch 1/5
907/907 [==============================] - 414s 455ms/step - loss: 1.0599 - accuracy: 0.6872
Epoch 2/5
907/907 [==============================] - 395s 436ms/step - loss: 0.1551 - accuracy: 0.9501
Epoch 3/5
907/907 [==============================] - 394s 435ms/step - loss: 0.0758 - accuracy: 0.9759
Epoch 4/5
907/907 [==============================] - 394s 435ms/step - loss: 0.0462 - accuracy: 0.9851
Epoch 5/5
907/907 [==============================] - 393s 433ms/step - loss: 0.0376 - accuracy: 0.9884


**Report accuracy of model 1**

In [ ]:
modelLoss , modelAccuracy = model.evaluate(X_test,Y_test)
print('Test Loss is %d'%modelLoss)
print('Test Accuracy is %d'%modelAccuracy)

1/1 [==============================] - 0s 374ms/step - loss: 0.0912 - accuracy: 1.0000
Test Loss is 0
Test Accuracy is 1


**Report accuracy of model 2**

In [ ]:
modelLoss_2 , modelAccuracy_2 = model_2.evaluate(X_test,Y_test)
print('Test Loss is %d'%modelLoss_2)
print('Test Accuracy is %d'%modelAccuracy_2)

1/1 [==============================] - 0s 216ms/step - loss: 0.0096 - accuracy: 1.0000
Test Loss is 0
Test Accuracy is 1


**Predict using first model**

In [ ]:
Y_predict=model.predict(X_test) 
classes_y=np.argmax(Y_predict,axis=1)

**Predict using second model**

In [ ]:
Y_predict_2=model_2.predict(X_test) 
classes_y_2=np.argmax(Y_predict_2,axis=1)

**Report accuracy,recall and precission**

In [ ]:
# calculate accuracy
accuracy = accuracy_score(Y_test, classes_y)
print('Model accuracy is: ', accuracy)

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", precision_score(Y_test, classes_y, average='micro'))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", recall_score(Y_test, classes_y, average='micro'))

Model accuracy is:  1.0
Precision: 1.0
Recall: 1.0


**Report accuracy,recall and precission**

In [ ]:
# calculate accuracy
accuracy_2 = accuracy_score(Y_test, classes_y_2)
print('Model accuracy is: ', accuracy_2)

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", precision_score(Y_test, classes_y_2, average='micro'))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", recall_score(Y_test, classes_y_2, average='micro'))

Model accuracy is:  1.0
Precision: 1.0
Recall: 1.0
